In [4]:
## Import modules
import pandas as pd
import numpy as np
from datetime import datetime

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm

import math
from sklearn.metrics import mean_squared_error

import random
import numpy as np

import os, warnings
warnings.filterwarnings('ignore')

In [2]:
# Forecasting and benchmarking
def benchmarking(df, var_name, result_file, no_exper = 10):
    end_index_of_df = len(df)
    
    lead_times = [1, 3, 6 , 12, 18, 24]
    previous_time = 24000
    
    no_of_experiments = no_exper

    text_file = open(result_file, "w")
    text_file.write("time, our, naive, average \n")

    for lead_time in lead_times:
        print('computing for lead time = ', str(lead_time), ' hours with history of ', str(previous_time), ' hours')
        
        rmse_array = []
        persist_array = []
        average_array = []

        for _ in range(no_of_experiments):
            offset = previous_time + lead_time
            start_index = random.randint(0, end_index_of_df - offset)
            #print(start_index)
            train = df[start_index:start_index+previous_time]
            test = df[start_index+previous_time:start_index+offset]

            y_hat_avg = test.copy()
            print('computation started')
          
            fit1 = ExponentialSmoothing(np.asarray(train[var_name]), seasonal_periods=240, trend='add', seasonal='add', ).fit()
            y_hat_avg['predicted'] = fit1.forecast(len(test))

            # persistence
            last_value = train[var_name].iloc[-1]
            y_hat_avg['naive'] = last_value * np.ones(len(test))

            # average
            mean_training_value = np.mean(train[var_name])
            y_hat_avg['aver'] = mean_training_value * np.ones(len(test))

            print('computation completed')

            # computing the error for exponential smoothing
            a = y_hat_avg[var_name]
            b = y_hat_avg['predicted']
            rmse_value = np.sqrt(np.mean((b - a) ** 2))
            rmse_array.append(rmse_value)

            # computing the error for persistence model
            a = y_hat_avg[var_name]
            b = y_hat_avg['naive']
            rmse_value = np.sqrt(np.mean((b - a) ** 2))
            persist_array.append(rmse_value)

            # computing the error for average model
            a = y_hat_avg[var_name]
            b = y_hat_avg['aver']
            rmse_value = np.sqrt(np.mean((b - a) ** 2))
            average_array.append(rmse_value)

        rmse_array = np.array(rmse_array)
        persist_array = np.array(persist_array)
        average_array = np.array(average_array)

        text_file.write("%s, %s, %s, %s \n" % (lead_time, np.mean(rmse_array), np.mean(persist_array), np.mean(average_array)))

    text_file.close()

In [ ]:
# read data
file = r'./data/mean_T_hourly.csv'
df = pd.read_csv(file, index_col='time').drop(['level'], axis=1)

#result save to comparison_50runs.txt file
res_file = r'./results/comparison_50runs.txt'

# call benchmarking function
benchmarking(df, 'Temperature', res_file, 50)